In [1]:

import psycopg2
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime, timedelta, time

# scaling 
from sklearn.preprocessing import StandardScaler

'\n# visualization\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nimport shap \n\n# dataset\nfrom sklearn.model_selection import train_test_split\nfrom torch.utils.data import Dataset, DataLoader, Sampler, BatchSampler, SequentialSampler\nfrom torch.cuda.amp import GradScaler, autocast\n\n# model \nimport torch\nimport torch.nn as nn\nimport torch.nn.functional as F\nimport torch.optim as optim\nimport lightgbm as lgb\nimport optuna\noptuna.logging.set_verbosity(optuna.logging.ERROR)\n\n# wandb\nimport wandb\n\n# evaluation\nfrom sklearn.metrics import log_loss, roc_auc_score, roc_curve, average_precision_score, accuracy_score, precision_recall_curve\nfrom sklearn.metrics import roc_auc_score, precision_recall_curve, auc as calculate_auc  \nfrom sklearn.model_selection import StratifiedKFold'

In [2]:
# PostgreSQL 서버 연결 정보

host = '192.168.0.76'  # 호스트 주소
database = 'eicu'  # 데이터베이스 이름
user = 'ykjeong'  # 사용자 이름
password = 'mdhi1234!'  # 비밀번호

# PostgreSQL 서버에 연결
conn = psycopg2.connect(
    dbname=database,
    user=user,
    password=password,
    host=host
)

## 데이터 추출

### patients

In [3]:
query = '''
SELECT
    patientunitstayid AS stay_id,
    gender,
    age,
    ethnicity AS race,
    unitType AS CU_type,
    unitAdmitTime24 AS intime, 
    unitdischargeoffset / 60 AS los, 
    unitDischargeTime24 AS death_or_dischtime,
    unitDischargeStatus AS dead_in_hosp
FROM
    eicu.patient
WHERE -- 1일 이상 60일 미만 입원한 19세 이상의 환자
    unitDischargeOffset / 1440.0 >= 1 
    AND unitDischargeOffset / 1440.0 < 60;
'''
patients = pd.read_sql_query(query, conn)
patients.head()

/tmp/ipykernel_1346879/4216977163.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  patients = pd.read_sql_query(query, conn)


,stay_id,gender,age,race,cu_type,intime,los,death_or_dischtime,dead_in_hosp
0,3181453,Male,25,Caucasian,CCU-CTICU,00:07:00,70,22:11:00,Alive
1,3181471,Male,53,Caucasian,SICU,23:09:00,66,17:17:00,Alive
2,3181495,Male,59,African American,Neuro ICU,09:23:00,109,22:50:00,Alive
3,3181506,Male,19,Caucasian,SICU,13:27:00,81,23:02:00,Alive
4,3181524,Male,> 89,Caucasian,Neuro ICU,10:08:00,326,00:47:00,Expired


In [4]:
patients.rename(columns={
    'cu_type':'CU type', 
    'dead_in_hosp':'dead in hosp',
    'death_or_dischtime':'death or dischtime'
}, inplace=True)

In [5]:
# 'age' 컬럼 핸들링 
# '> 89'를 91로 처리
patients.loc[patients['age'] == '> 89', 'age'] = 91

# 결측치를 평균으로 대체
patients['age'] = pd.to_numeric(patients['age'], errors='coerce')
patients['age'] = patients['age'].fillna(patients['age'].mean())
patients['age'] = patients['age'].astype(int)

# 18세 이상만 추출 
patients = patients[patients['age'] > 18]

In [6]:
def categorize_careunit(careunit):
    if 'MICU' in careunit or'Med-Surg ICU' in careunit:
        return 'MICU'
    elif 'SICU' in careunit:
        return 'SICU'
    elif 'Neuro ICU' in careunit:
        return 'Neuro ICU'
    elif 'Cardiac ICU' in careunit or 'CSICU' in careunit:
        return 'CCU'
    elif 'CVICU' in careunit:
        return 'CVICU'
    else:
        return 'other'

patients['CU type'] = patients['CU type'].apply(categorize_careunit)
patients['CU type'].unique()

array(['other', 'SICU', 'Neuro ICU', 'MICU', 'CCU'], dtype=object)

In [7]:
# 인종 분류 및 결측치 수정 

def categorize_race(race):
    if 'Caucasian' in race:
        return 'white'
    elif 'African American' in race:
        return 'black'
    elif 'Hispanic' in race:
        return 'hispanic/latino'
    elif 'ASIAN' in race:
        return 'asian'
    else:
        return 'other'

patients['race'] = patients['race'].apply(categorize_race)
patients['race'].unique()

array(['white', 'black', 'hispanic/latino', 'other'], dtype=object)

In [8]:
# dead in hosp 컬럼의 값을 변경하고 비율 확인

patients.loc[:, 'dead in hosp'] = patients['dead in hosp'].map(lambda x: 1 if x == 'Expired' else 0)
patients['dead in hosp'].value_counts(normalize=True)

dead in hosp
0    0.947866
1    0.052134
Name: proportion, dtype: float64

In [344]:
# stay_id 분리

patients_pos_samp = patients[patients['dead in hosp'] == 1]

# 'dead in hosp'가 0인 행들에서 50%만 랜덤으로 추출
patients_neg_samp = patients[patients['dead in hosp'] == 0].sample(frac=0.2, random_state=42)

patients = pd.concat([patients_neg_samp, patients_pos_samp])
patients['dead in hosp'].value_counts(normalize=True)

dead in hosp
0    0.784312
1    0.215688
Name: proportion, dtype: float64

In [9]:
patients.to_parquet('eICU_patients.parquet')

### lab

In [10]:
query = '''
SELECT 
    lab.patientunitstayid AS stay_id,
    lab.labname AS label,
    lab.labresulttext AS value,
    lab.labTypeID AS labtype, 
    lab.labmeasurenamesystem AS valueoum, 
    lab.labresultoffset / 60 AS time
FROM
    eicu.lab lab
WHERE(
        lab.labname LIKE '%AST%' OR
        lab.labname LIKE '%ALT%' OR
        lab.labname LIKE '%albumin%' OR
        lab.labname LIKE '%bilirubin%' OR
        lab.labname LIKE '%BUN%' OR
        lab.labname LIKE '%chloride%' OR
        lab.labname LIKE '%CRP%' OR
        lab.labname LIKE '%glucose%' OR
        lab.labname LIKE '%Hgb%' OR
        lab.labname LIKE '%respiratory%' OR
        lab.labname LIKE '%platelet%' OR
        lab.labname LIKE '%potassium%' OR
        lab.labname LIKE '%Temperature%' OR
        lab.labname LIKE '%urinary sodium%' OR
        lab.labname LIKE '%urinary creatinine%' OR
        lab.labname LIKE '%WBC x 1000%' OR
        lab.labname LIKE '%PT%' AND 
        lab.labmeasurenamesystem LIKE '%sec%' 
    )
    AND lab.patientunitstayid IN (
        SELECT
            patient.patientunitstayid
        FROM
            eicu.patient patient
        WHERE
            patient.unitDischargeOffset / 1440.0 >= 1 
            AND patient.unitDischargeOffset / 1440.0 < 60
    );
'''
lab = pd.read_sql_query(query, conn)
lab = lab[lab['stay_id'].isin(patients['stay_id'])]

/tmp/ipykernel_1346879/2683277839.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  lab = pd.read_sql_query(query, conn)


In [11]:
# lab['label']의 값이 'glucose - CSF', 'prealbumin', 'direct bilirubin', 'CRP-hs'인 행 삭제 

values_to_remove = ['glucose - CSF', 'prealbumin', 'direct bilirubin', 'CRP-hs', 'PTT']
lab = lab[~lab['label'].isin(values_to_remove)]

In [12]:
lab['label'] = lab['label'].replace({
    r' \(SGPT\)': '',
    r' \(SGOT\)': '',
    r'urinary ': '',
    r' x 1000': '',
    r'bedside ': '',
    r'total ': ''
}, regex=True)

In [13]:
# value 열의 모든 특수문자를 제거하고 수치형으로 변환

lab['value'] = lab['value'].str.replace(r'[^\w\s]', '', regex=True)
lab['value'] = pd.to_numeric(lab['value'], errors='coerce')

In [14]:
# vital sign 이상치 조정 

ranges = {           
    'ALT':(0, 5000), 
    'AST':(0, 10000), 
    'albumin':(0, 5.5), 
    'BUN':(0, 150), 
    'bilirubin':(0, 50),
    'CRP':(0, 300), 
    'chloride':(70, 135),
    'creatinine':(0, 15),
    'glucose':(0, 600),
    'Hgb':(0, 25),
    'Temperature': (32, 41), 
    'potassium':(2, 9),
    'sodium':(105, 170),
    'platelets':(0, 1000),
    'PT':(0, 8), 
    'WBC':(0, 90)}

# 범위를 벗어난 값들을 NaN으로 대체
def replace_out_of_range_with_nan(row):
    label = row['label']
    value = row['value']
    
    if label in ranges:
        lower, upper = ranges[label]
        
        # 값이 범위를 벗어나면 NaN으로 대체
        if value < lower or value > upper:
            return np.nan
    return value

lab['value'] = lab.apply(replace_out_of_range_with_nan, axis=1)

# NaN 값을 각 label에 대한 평균값으로 대체
lab['value'] = lab.groupby('label')['value'].transform(lambda x: x.fillna(x.mean()))
lab.head()

,stay_id,label,value,labtype,valueoum,time
0,3181453,glucose,130.0,1.0,mg/dL,106
1,3181453,PT,NaN,3.0,sec,2
3,3181453,AST,27.0,1.0,Units/L,106
4,3181453,chloride,99.0,1.0,mmol/L,33
6,3181453,ALT,11.0,1.0,Units/L,2


In [15]:
lab.to_parquet('eICU_lab.parquet')

### Vital aperiodic: sbp, dbp

In [16]:
query = '''
SELECT
    vitala.patientunitstayid AS stay_id,
    vitala.observationoffset / 60 AS time, 
    vitala.noninvasivesystolic AS sbp,
    vitala.noninvasivediastolic AS dbp
FROM
    eicu.vitalaPeriodic vitala
WHERE
    vitala.patientUnitStayID IN (
        SELECT
            patient.patientunitstayid
        FROM
            eicu.patient patient
        WHERE
            patient.unitDischargeOffset / 1440.0 >= 1 
            AND patient.unitDischargeOffset / 1440.0 < 60
    );
'''
vitalaPeriodic = pd.read_sql_query(query, conn)
vitalaPeriodic = vitalaPeriodic[vitalaPeriodic['stay_id'].isin(patients['stay_id'])]

/tmp/ipykernel_1346879/1725982474.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vitalaPeriodic = pd.read_sql_query(query, conn)


In [17]:
vitalap = pd.melt(vitalaPeriodic, id_vars=['stay_id', 'time'], value_vars=['sbp', 'dbp'],
                    var_name='label', value_name='value')
vitalap = vitalap[['stay_id', 'label', 'value', 'time']]

In [18]:
# vital sign 이상치 조정 

# 기준 설정
ranges = {
    'sbp': (40, 230),
    'dbp': (20, 130)
}


# 범위를 벗어난 값들을 NaN으로 대체
def replace_out_of_range_with_nan(row):
    label = row['label']
    value = row['value']
    
    if label in ranges:
        lower, upper = ranges[label]
        
        # 값이 범위를 벗어나면 NaN으로 대체
        if value < lower or value > upper:
            return np.nan
    return value

vitalap['value'] = vitalap.apply(replace_out_of_range_with_nan, axis=1)
vitalap['value'] = vitalap.groupby('label')['value'].transform(lambda x: x.fillna(x.mean()))

In [19]:
vitalap.to_parquet('eICU_vitalap.parquet')

### Nurse charting : Respiratory Rate, Heart rate, sbp, dbp, temperature, SpO2

In [4]:
query = '''
SELECT 
    chart.patientunitstayid AS stay_id,
    chart.nursingChartCellTypeValName AS label,
    chart.nursingChartValue AS value,
    chart.nursingChartEntryOffset / 60 AS time
FROM
    eicu.nurseCharting chart
WHERE (
        chart.nursingChartCellTypeValName LIKE '%Heart Rate%' OR
        chart.nursingChartCellTypeValName LIKE '%Respiratory Rate%' OR
        chart.nursingChartCellTypeValName LIKE '%Non-Invasive BP Diastolic%' OR
        chart.nursingChartCellTypeVallabel LIKE '%Temperature (%' OR
        chart.nursingChartCellTypeValName LIKE '%Saturation%' 
    )
    AND chart.patientunitstayid IN (
        SELECT
            patient.patientunitstayid
        FROM
            eicu.patient patient
        WHERE
            patient.unitDischargeOffset / 1440.0 >= 1 
            AND patient.unitDischargeOffset / 1440.0 < 60
    );
'''
chart = pd.read_sql_query(query, conn)
chart = chart[chart['stay_id'].isin(patients['stay_id'])]

/tmp/ipykernel_1368314/2897185212.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chart = pd.read_sql_query(query, conn)


In [5]:
chart['label'] = chart['label'].replace({'Non-Invasive BP Systolic':'sbp',
                                           'Non-Invasive BP Diastolic':'dbp',
                                           'O2 Saturation':'SpO2'})

In [6]:
# 화씨 온도를 섭씨 온도로 변환
fahrenheit_mask = chart['label'] == 'Temperature (F)'
celcius_mask = chart['label'] == 'Temperature (C)'
chart.loc[fahrenheit_mask, 'value'] = (chart.loc[fahrenheit_mask, 'value'] - 32) * 5.0 / 9.0

# label 수정
chart.loc[fahrenheit_mask, 'label'] = 'Temperature'
chart.loc[celcius_mask, 'label'] = 'Temperature'

In [3]:
chart = pd.read_parquet('eICU_chart.parquet')
patients = pd.read_parquet('eICU_patients.parquet')

In [4]:
# vital sign 이상치 조정 

chart['value'].replace('', np.nan, inplace=True)
chart['value'] = chart['value'].astype(float)

# 기준 설정
ranges = {
    'Respiratory Rate': (5, 50),
    'Heart Rate': (10, 190),
    'sbp': (40, 230),
    'dbp': (20, 130),
    'Temperature': (32, 41), 
    'SpO2': (68, 100)
}
# 각 label에 대한 평균값 계산
mean_values = chart.groupby('label')['value'].mean()

# 범위를 벗어난 값들을 NaN으로 대체
def replace_out_of_range_with_nan(row):
    label = row['label']
    value = row['value']
    
    if label in ranges:
        lower, upper = ranges[label]
        
        # 값이 범위를 벗어나면 NaN으로 대체
        if value < lower or value > upper:
            return np.nan
    return value

chart['value'] = chart.apply(replace_out_of_range_with_nan, axis=1)
chart['value'] = chart.groupby('label')['value'].transform(lambda x: x.fillna(x.mean()))
chart.head()

/tmp/ipykernel_1369658/338451565.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  chart['value'].replace('', np.nan, inplace=True)


,stay_id,label,value,time
0,3181453,SpO2,92.0,63
1,3181453,dbp,70.0,26
2,3181453,Respiratory Rate,19.0,35
3,3181453,Respiratory Rate,25.0,7
4,3181453,Heart Rate,111.0,42


In [5]:
chart.to_parquet('eICU_chart.parquet')

### physical Exam: GCS 

In [6]:
query = '''
SELECT 
    exam.patientUnitStayID AS stay_id,
    exam.physicalexampath AS label,
    exam.physicalExamOffset / 60 AS time,
    exam.physicalExamText AS value
FROM
    eicu.physicalExam exam
WHERE
    exam.physicalexampath LIKE '%Eyes Score%' OR
    exam.physicalexampath LIKE '%Verbal Score%' OR
    exam.physicalexampath LIKE '%Motor Score%';
'''
exam = pd.read_sql_query(query, conn)
exam = exam[exam['stay_id'].isin(patients['stay_id'])]

/tmp/ipykernel_1369658/2141619851.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  exam = pd.read_sql_query(query, conn)


In [7]:
exam['label'] = exam['label'].replace(r'notes/Progress Notes/Physical Exam/Physical Exam/Neurologic/GCS/', '', regex=True)
exam['label'] = exam['label'].str.replace(r'[^a-zA-Z\s]', '', regex=True)
exam['value'] = pd.to_numeric(exam['value'], errors='coerce')

In [8]:
# vital sign 이상치 조정 
    
ranges = {           
    'Eyes Score':(1, 4),
    'Verbal Score':(1, 5),
    'Motor Score':(1, 6)}

# 각 label에 대한 평균값 계산
mean_values = exam.groupby('label')['value'].mean()

# 범위를 벗어난 값들을 NaN으로 대체
def replace_out_of_range_with_nan(row):
    label = row['label']
    value = row['value']
    
    if label in ranges:
        lower, upper = ranges[label]
        
        # 값이 범위를 벗어나면 NaN으로 대체
        if value < lower or value > upper:
            return np.nan
    return value

exam['value'] = exam.apply(replace_out_of_range_with_nan, axis=1)
exam['value'] = exam.groupby('label')['value'].transform(lambda x: x.fillna(x.mean()))
exam.head()

,stay_id,label,time,value
0,176895,Eyes Score,0,3
1,176895,Motor Score,0,1
2,176895,Verbal Score,0,2
3,149700,Motor Score,0,1
4,149700,Verbal Score,0,1


In [9]:
exam.to_parquet('eICU_exam.parquet')

## concat, pivot and merge 

In [10]:
chart =  pd.read_parquet('eICU_chart.parquet')
lab =  pd.read_parquet('eICU_lab.parquet')
exam =  pd.read_parquet('eICU_exam.parquet')
vitalap =  pd.read_parquet('eICU_vitalap.parquet')

In [3]:
df = pd.concat([chart, lab, exam, vitalap])
df.drop(columns='valueoum', axis=1, inplace=True)

In [4]:
set(df['stay_id'].unique()) - set(patients['stay_id'].unique())

set()

In [5]:
df = df[df['time'] >= 0]

In [6]:
df.to_parquet('eICU_df.parquet')

In [2]:
df = pd.read_parquet('eICU_df.parquet')
patients = pd.read_parquet('eICU_patients.parquet')

In [3]:
df = df.pivot_table(index=['stay_id', 'time'], columns='label', values='value', aggfunc='mean')
df.reset_index(inplace=True)

In [4]:
df_chart = df.merge(patients, on='stay_id', how='inner') 
df_chart.head()

,stay_id,time,ALT,AST,BUN,CRP,Eyes Score,Heart Rate,Hgb,Motor Score,...,sbp,sodium,gender,age,race,CU type,intime,los,death or dischtime,dead in hosp
0,141168,0,NaN,NaN,NaN,NaN,NaN,105.0,NaN,NaN,...,NaN,NaN,Female,70,white,MICU,15:54:00,59,03:50:00,1
1,141168,1,NaN,NaN,NaN,NaN,4.0,140.0,NaN,6.0,...,NaN,NaN,Female,70,white,MICU,15:54:00,59,03:50:00,1
2,141168,2,NaN,NaN,NaN,NaN,NaN,140.0,NaN,NaN,...,108.5,NaN,Female,70,white,MICU,15:54:00,59,03:50:00,1
3,141168,3,NaN,NaN,NaN,NaN,NaN,134.0,NaN,NaN,...,NaN,NaN,Female,70,white,MICU,15:54:00,59,03:50:00,1
4,141168,4,NaN,NaN,NaN,NaN,NaN,134.0,NaN,NaN,...,NaN,NaN,Female,70,white,MICU,15:54:00,59,03:50:00,1


In [5]:
df_chart['dead in hosp'].value_counts(normalize=True)

dead in hosp
0    0.933042
1    0.066958
Name: proportion, dtype: float64

## cleansing and fillna
- 결측치 처리 
- 컬럼명/순서 수정
- 날짜 컬럼 삭제

In [6]:
# 각 컬럼의 결측치 비율 계산
    
def missing_value_ratio(df):
    missing_ratio = df.isna().mean() * 100
    
    # 결측치 비율을 오름차순으로 정렬하여 출력)
    print(missing_ratio.sort_values(ascending=True))

missing_value_ratio(df_chart)

stay_id                0.000000
los                    0.000000
intime                 0.000000
CU type                0.000000
race                   0.000000
age                    0.000000
gender                 0.000000
death or dischtime     0.000000
dead in hosp           0.000000
time                   0.000000
dbp                   18.570550
sbp                   29.149157
Heart Rate            35.493843
Respiratory Rate      39.393334
SpO2                  45.741889
glucose               75.759590
potassium             92.182777
chloride              93.225824
BUN                   93.274466
Hgb                   93.298798
platelets             94.004522
WBC                   94.060916
albumin               97.573669
AST                   97.965989
ALT                   97.979684
bilirubin             98.003077
Motor Score           98.648463
Eyes Score            98.648485
Verbal Score          98.648530
Temperature           99.374319
sodium                99.907434
creatini

In [31]:
# stay_id별로 결측치 처리 

def nan_fill(df):
    # 1. forward fill
    # 일부 컬럼은 제외 
    columns_to_exclude = ['stay_id', 'gender', 'age', 'race', 'CU type', 'intime', 'death or dischtime', 'charttime','dead in hosp', 'time']
    selected_columns = df.columns.difference(columns_to_exclude)
    df_ffill = df.copy()
    df_ffill[selected_columns] = df.groupby('stay_id')[selected_columns].apply(lambda group: group.ffill()).reset_index(drop=True)
    
    # 2. median fill
    overall_median_values = df_ffill[selected_columns].median()
    df_ffill[selected_columns] = df_ffill[selected_columns].fillna(overall_median_values)

    return df_ffill

df_fillna = nan_fill(df_chart)

In [4]:
df_fillna['dead in hosp'].value_counts(normalize=True)

dead in hosp
0    0.933042
1    0.066958
Name: proportion, dtype: float64

In [5]:
df_fillna['intime'] = pd.to_datetime(df_fillna['intime'])
df_fillna['charttime'] = df_fillna['intime'] + pd.to_timedelta(df_fillna['time'], unit='h')
df_fillna['death or dischtime'] = df_fillna['intime'] + pd.to_timedelta(df_fillna['los'], unit='h')

/tmp/ipykernel_1699487/4098409949.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_fillna['intime'] = pd.to_datetime(df_fillna['intime'])


In [3]:
# pos와 neg 분리 

df_pos = df_fillna[df_fillna['dead in hosp'] == 1]
df_neg = df_fillna[df_fillna['dead in hosp'] == 0]

In [4]:
def resample_group(group):
    # 'charttime'을 datetime 인덱스로 설정
    group = group.set_index('charttime')
    
    # float형 데이터만 선택하여 평균 계산
    numeric_cols = group.select_dtypes(include=['float']).columns
    group_resampled = group[numeric_cols].resample('h').mean()
    
    # int, object형 데이터는 해당 stay_id의 전후값으로 계산
    non_numeric_cols = group.select_dtypes(exclude=['float']).columns
    group_non_numeric_resampled = group[non_numeric_cols].resample('h').ffill().bfill()
    
    # 리샘플링된 numeric 데이터와 non-numeric 데이터를 병합
    group_resampled = group_resampled.join(group_non_numeric_resampled)
    
    # stay_id 다시 설정
    group_resampled['stay_id'] = group['stay_id'].iloc[0]
    
    return group_resampled.reset_index()

def resampling(df):
    resampled = df.groupby('stay_id').apply(resample_group).reset_index(drop=True)
    resampled.sort_values(by=['stay_id', 'charttime'], ascending=True, inplace=True)
    return resampled

# 리샘플링 함수 호출
df_resample_pos = resampling(df_pos)
df_resample_pos.head()

,charttime,ALT,AST,BUN,CRP,Eyes Score,Heart Rate,Hgb,Motor Score,Respiratory Rate,...,stay_id,time,gender,age,race,CU type,intime,los,death or dischtime,dead in hosp
0,2024-08-30 15:00:00,28.0,32.0,24.0,92.55513,4.0,105.0,18.62,6.0,19.0,...,141168,0.0,Female,70.0,white,MICU,2024-08-30 15:54:00,59.0,2024-09-02 02:54:00,1.0
1,2024-08-30 16:00:00,28.0,32.0,24.0,92.55513,4.0,140.0,18.62,6.0,19.0,...,141168,0.0,Female,70.0,white,MICU,2024-08-30 15:54:00,59.0,2024-09-02 02:54:00,1.0
2,2024-08-30 17:00:00,28.0,32.0,24.0,92.55513,4.0,140.0,18.62,6.0,19.0,...,141168,1.0,Female,70.0,white,MICU,2024-08-30 15:54:00,59.0,2024-09-02 02:54:00,1.0
3,2024-08-30 18:00:00,28.0,32.0,24.0,92.55513,4.0,134.0,18.62,6.0,19.0,...,141168,2.0,Female,70.0,white,MICU,2024-08-30 15:54:00,59.0,2024-09-02 02:54:00,1.0
4,2024-08-30 19:00:00,28.0,32.0,24.0,92.55513,4.0,134.0,18.62,6.0,19.0,...,141168,3.0,Female,70.0,white,MICU,2024-08-30 15:54:00,59.0,2024-09-02 02:54:00,1.0


In [16]:
# neg는 데이터세트가 많으므로 최근 행을 먼저 추출하고 리샘플링을 진행

# stay_id별로 가장 최근 행만 먼저 추출
df_neg_max_time = df_neg.loc[df_neg.groupby('stay_id')['charttime'].idxmax()]

# Calculate the difference in hours
df_neg_max_time['time_diff'] = (df_neg_max_time['death or dischtime'] - df_neg_max_time['charttime']).dt.total_seconds() / 3600
filtered_df = df_neg_max_time[(df_neg_max_time['time_diff'] >= 0) & (df_neg_max_time['time_diff'] <= 1)]
df_neg_filtered = df_neg[df_neg['stay_id'].isin(filtered_df['stay_id'].unique())]

In [5]:
# stay_id별로 가장 최근 행만 먼저 추출
df_resample_pos_max_time = df_resample_pos.loc[df_resample_pos.groupby('stay_id')['charttime'].idxmax()]

# Calculate the difference in hours
df_resample_pos_max_time['time_diff'] = (df_resample_pos_max_time['death or dischtime'] - df_resample_pos_max_time['charttime']).dt.total_seconds() / 3600
filtered_df = df_resample_pos_max_time[(df_resample_pos_max_time['time_diff'] >= 0) & (df_resample_pos_max_time['time_diff'] <= 1)]
df_resample_pos_filtered = df_resample_pos[df_resample_pos['stay_id'].isin(filtered_df['stay_id'].unique())]

In [17]:
df_neg_filtered.to_parquet('eICU_df_neg_filtered.parquet')

In [19]:
df_resample_neg = resampling(df_neg_filtered)
df_resample_neg.head()

,charttime,ALT,AST,BUN,CRP,Eyes Score,Heart Rate,Hgb,Motor Score,Respiratory Rate,...,stay_id,time,gender,age,race,CU type,intime,los,death or dischtime,dead in hosp
0,2024-08-30 12:00:00,28.0,32.0,26.0,92.55513,3.0,104.101516,18.62,6.0,33.253783,...,141227,0.0,Male,82.0,white,MICU,2024-08-30 12:03:00,27.0,2024-08-31 15:03:00,0.0
1,2024-08-30 13:00:00,28.0,32.0,26.0,92.55513,3.0,102.500000,18.62,6.0,32.000000,...,141227,1.0,Male,82.0,white,MICU,2024-08-30 12:03:00,27.0,2024-08-31 15:03:00,0.0
2,2024-08-30 14:00:00,28.0,32.0,26.0,92.55513,3.0,106.800000,18.62,6.0,32.800000,...,141227,2.0,Male,82.0,white,MICU,2024-08-30 12:03:00,27.0,2024-08-31 15:03:00,0.0
3,2024-08-30 15:00:00,28.0,32.0,26.0,92.55513,3.0,98.400000,18.62,6.0,27.000000,...,141227,3.0,Male,82.0,white,MICU,2024-08-30 12:03:00,27.0,2024-08-31 15:03:00,0.0
4,2024-08-30 16:00:00,28.0,32.0,26.0,92.55513,3.0,104.000000,18.62,6.0,28.666667,...,141227,4.0,Male,82.0,white,MICU,2024-08-30 12:03:00,27.0,2024-08-31 15:03:00,0.0


In [21]:
# stay_id별 데이터 개수 계산
group_counts = df_resample_neg.groupby('stay_id').size()
valid_stay_ids = group_counts[group_counts >= 24].index

# 원래 데이터프레임에서 24개 이상의 데이터를 가지는 stay_id 그룹만 선택
df_fillna_neg_24 = df_resample_neg[df_resample_neg['stay_id'].isin(valid_stay_ids)]
df_fillna_neg_24.sort_values(by=['stay_id', 'charttime'], inplace=True)

# stay_id별 데이터 개수 계산
group_counts = df_resample_pos_filtered.groupby('stay_id').size()
valid_stay_ids = group_counts[group_counts >= 24].index

# 원래 데이터프레임에서 24개 이상의 데이터를 가지는 stay_id 그룹만 선택
df_fillna_pos_24 = df_resample_pos_filtered[df_resample_pos_filtered['stay_id'].isin(valid_stay_ids)]
df_fillna_pos_24.sort_values(by=['stay_id', 'charttime'], inplace=True)

/tmp/ipykernel_1735805/62225354.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fillna_neg_24.sort_values(by=['stay_id', 'charttime'], inplace=True)


In [22]:
# 모든 stay_id group이 1시간 간격의 데이터를 가지고 있는지 확인

def check_hourly_intervals(df):
    groups_with_problems = []

    for stay_id, group in df.groupby('stay_id'):
        group = group.sort_values('charttime')
        time_diffs = group['charttime'].diff().dropna()
        
        if not all(time_diffs == pd.Timedelta(hours=1)):
            groups_with_problems.append(stay_id)
    
    return groups_with_problems

len(check_hourly_intervals(df_fillna_pos_24)) == 0
len(check_hourly_intervals(df_fillna_neg_24)) == 0

True

In [10]:
# positive의 경우 각 stay_id별로 최신 시간을 기준으로 이전 24개의 행만 남김

def keep_last_24_hours(df):
    # 최신 charttime을 기준으로 정렬
    df_sorted = df.sort_values(by='charttime', ascending=True)
    df_kept = df_sorted.groupby('stay_id').tail(24)
    df_kept = df_kept.sort_values(by=['stay_id','charttime'], ascending=True)
    
    return df_kept

df_pos_24 = keep_last_24_hours(df_fillna_pos_24)
len(check_hourly_intervals(df_pos_24)) == 0

In [23]:
# negative의 경우 각 stay_id에서 일부 시점을 무작위로 선택해 해당 시점 전후의 값만 사용

def keep_random_24_hours(df):
    np.random.seed(42)  # 시드 고정
    
    def select_random_window(group):
        group_sorted = group.sort_values(by='charttime', ascending=False)
        random_index = np.random.randint(0, len(group_sorted))

        start_index = max(0, random_index - 11)
        end_index = min(len(group_sorted), random_index + 12 + 1)
        selected_window = group_sorted.iloc[start_index:end_index]

        if len(selected_window) < 24:
            if start_index == 0:
                additional_rows = group_sorted.iloc[end_index:end_index + (24 - len(selected_window))]
            else:
                additional_rows = group_sorted.iloc[max(0, start_index - (24 - len(selected_window))):start_index]
            selected_window = pd.concat([selected_window, additional_rows]).drop_duplicates().head(24)

        return selected_window
    
    # 각 stay_id 그룹에 대해 select_random_window 함수를 적용
    df_kept = df.groupby('stay_id').apply(select_random_window).reset_index(drop=True)
    df_kept = df_kept.sort_values(by=['stay_id','charttime'], ascending=True)
    
    return df_kept

df_neg_24 = keep_random_24_hours(df_fillna_neg_24)

In [24]:
len(df_neg_24), len(df_pos_24)

(635592, 60336)

In [128]:
# neg 비율을 언더샘플링 > neg:pos 비율을 조정

neg_undersampling = df_neg_24['stay_id'].drop_duplicates().sample(frac=1, random_state=42)
df_neg_24_under = df_neg_24[df_neg_24['stay_id'].isin(neg_undersampling)]

df_24 = pd.concat([df_neg_24_under, df_pos_24])
df_24.sort_values(by=['stay_id'], ascending=True, inplace=True)
df_24['dead in hosp'].value_counts(normalize=True)

dead in hosp
0.0    0.913301
1.0    0.086699
Name: proportion, dtype: float64

In [129]:
df_24_fillna = nan_fill(df_24)
len(check_hourly_intervals(df_24_fillna)) == 0

True

In [130]:
df_24_fillna.sort_values(by=['stay_id', 'charttime'], ascending=True, inplace=True)

In [131]:
df_24_fillna.columns

Index(['charttime', 'ALT', 'AST', 'BUN', 'CRP', 'Eyes Score', 'Heart Rate',
       'Hgb', 'Motor Score', 'Respiratory Rate', 'SpO2', 'Temperature',
       'Verbal Score', 'WBC', 'albumin', 'bilirubin', 'chloride', 'creatinine',
       'dbp', 'glucose', 'platelets', 'potassium', 'sbp', 'sodium', 'stay_id',
       'time', 'gender', 'age', 'race', 'CU type', 'intime', 'los',
       'death or dischtime', 'dead in hosp'],
      dtype='object')

In [132]:
# datetime 변수 핸들링
    
def datetime_remove(df):
    df_no_dates = df.drop(columns=['intime', 'death or dischtime'])
    df_no_dates = df_no_dates.sort_values(by=['stay_id', 'charttime'], ascending=True)
    return df_no_dates

df_24 = datetime_remove(df_24_fillna)

### 원핫인코딩

In [133]:
df_24 = df_24[df_24['gender'] != '']

In [134]:
# 범주형 열 목록
categorical_cols = ['race', 'gender', 'CU type']

# 범주형 변수 원핫인코딩
df_encoded = pd.get_dummies(df_24, columns=categorical_cols)
df_encoded['dead in hosp'].value_counts(normalize=True)

dead in hosp
0.0    0.913248
1.0    0.086752
Name: proportion, dtype: float64

### 데이터 개형 수정
- 더미 데이터 추가
- 컬럼 순서 재정렬

In [135]:
df_encoded.columns

Index(['charttime', 'ALT', 'AST', 'BUN', 'CRP', 'Eyes Score', 'Heart Rate',
       'Hgb', 'Motor Score', 'Respiratory Rate', 'SpO2', 'Temperature',
       'Verbal Score', 'WBC', 'albumin', 'bilirubin', 'chloride', 'creatinine',
       'dbp', 'glucose', 'platelets', 'potassium', 'sbp', 'sodium', 'stay_id',
       'time', 'age', 'los', 'dead in hosp', 'race_black',
       'race_hispanic/latino', 'race_other', 'race_white', 'gender_Female',
       'gender_Male', 'gender_Other', 'gender_Unknown', 'CU type_CCU',
       'CU type_MICU', 'CU type_Neuro ICU', 'CU type_SICU', 'CU type_other'],
      dtype='object')

In [136]:
df_encoded['CU type_CVICU'] = False
df_encoded['race_asian'] = False
df_encoded['PT'] = 4.0
df_encoded = df_encoded[df_encoded['CU type_Neuro ICU'] == False]

In [137]:
df_encoded = df_encoded[['charttime', 'stay_id', 'ALT', 'AST', 'albumin', 'BUN', 'CRP',
       'chloride', 'creatinine', 'Eyes Score', 'Motor Score',
       'Verbal Score', 'glucose', 'Heart Rate', 'Hgb',
       'platelets', 'potassium', 'PT', 'Respiratory Rate',
       'sodium', 'SpO2', 'Temperature', 'WBC', 'dbp', 'sbp', 'age', 
       'race_asian', 'race_black', 'race_hispanic/latino',
       'race_other', 'race_white', 'gender_Female', 'gender_Male', 'CU type_CCU',
       'CU type_CVICU', 'CU type_MICU', 'CU type_SICU',
       'dead in hosp']]

#### X, y split, 정규화

In [138]:
# vital sign 이상치 조정 

# 기준 설정
ranges = {
    'Respiratory Rate': (5, 50),
    'Heart Rate': (10, 190),
    'sbp': (40, 230),
    'dbp': (20, 130),
    'Temperature': (32, 41), 
    'SpO2': (68, 100),    
    'GCS Eye Opening':(1, 4),
    'GCS Verbal Response':(1, 5),
    'GCS Motor Response':(1, 6),
        
    'ALT':(0, 5000), 
    'AST':(0, 10000), 
    'Albumin':(0, 5.5), 
    'BUN':(0, 150), 
    'Bilirubin':(0, 50),
    'CRP':(0, 300), 
    'Chloride':(70, 135),
    'Creatinine':(0, 15),
    'Glucose':(0, 600),
    'Hemoglobin':(0, 25),
    'Potassium':(2, 9),
    'Sodium':(105, 170),
    'Platelet':(0, 1000),
    'Prothrombin time':(0, 8), 
    'WBC':(0, 90), 
}
# 각 열에 대해 이상치를 NaN으로 대체
for col, (lower, upper) in ranges.items():
    if col in df_encoded.columns:
        df_encoded[col] = np.where((df_encoded[col] < lower) | (df_encoded[col] > upper), np.nan, df_encoded[col])

# NaN 값을 각 열에 대한 평균값으로 대체
for col in ranges.keys():
    if col in df_encoded.columns:
        df_encoded[col] = df_encoded[col].fillna(df_encoded[col].mean())

df_encoded.head()

,charttime,stay_id,ALT,AST,albumin,BUN,CRP,chloride,creatinine,Eyes Score,...,race_hispanic/latino,race_other,race_white,gender_Female,gender_Male,CU type_CCU,CU type_CVICU,CU type_MICU,CU type_SICU,dead in hosp
23,2024-08-30 16:00:00,141227,28.0,32.0,3.75,30.0,92.55513,101.0,10.842593,3.0,...,False,False,True,False,True,False,False,True,False,0.0
22,2024-08-30 17:00:00,141227,28.0,32.0,3.75,26.0,92.55513,101.0,10.842593,3.0,...,False,False,True,False,True,False,False,True,False,0.0
21,2024-08-30 18:00:00,141227,28.0,32.0,3.75,26.0,92.55513,101.0,10.842593,3.0,...,False,False,True,False,True,False,False,True,False,0.0
20,2024-08-30 19:00:00,141227,28.0,32.0,3.75,26.0,92.55513,101.0,10.842593,3.0,...,False,False,True,False,True,False,False,True,False,0.0
19,2024-08-30 20:00:00,141227,28.0,32.0,3.75,26.0,92.55513,101.0,10.842593,3.0,...,False,False,True,False,True,False,False,True,False,0.0


In [139]:
y_val = df_encoded['dead in hosp']
X_val = df_encoded.drop(columns=['dead in hosp'], inplace=False)

In [140]:
X_val.shape, y_val.shape

((643176, 37), (643176,))

In [141]:
def scaling(df):
    # 'charttime'을 인덱스로 설정
    df.set_index('charttime', inplace=True)
    df.drop(columns=['stay_id'], inplace=True)
    
    # 정규화
    scaler = StandardScaler() # MinMaxScaler / StandardScaler
    df_scaled = scaler.fit_transform(df)
    
    return df_scaled

X_val_scaled = scaling(X_val)

In [142]:
def class_ratio(df):
    # 클래스별 개수 계산
    class_counts = df.value_counts()

    # 전체 데이터 개수
    total_count = len(df)

    # 클래스별 비율 계산
    class_ratios = class_counts / total_count

    print("클래스 0의 비율:", class_ratios[0])
    print("클래스 1의 비율:", class_ratios[1])

# 비율 유지되는지 확인 

class_ratio(df_encoded['dead in hosp'])
class_ratio(y_val)

클래스 0의 비율: 0.9117131236240158
클래스 1의 비율: 0.08828687637598417
클래스 0의 비율: 0.9117131236240158
클래스 1의 비율: 0.08828687637598417


In [143]:
num_features = X_val_scaled.shape[1]
num_features

35

In [144]:
df_encoded.to_parquet('eICU_valid_dataset.parquet')